In [131]:
import pandas as pd
import numpy as np
import os
import openpyxl

In [339]:
# Files Directions, Rutes and file names
filepathCostos = r'\\u-srvnas\Areas\Mantenimiento\00.1 Mantto Planeamiento\0000. Gestión 2019\1. REPORTES\1. BASES DE DATOS DIARIOS\Data SAP\01. Costos'
filepathOtAv = r'\\u-srvnas\Areas\Mantenimiento\00.1 Mantto Planeamiento\0000. Gestión 2019\1. REPORTES\1. BASES DE DATOS DIARIOS\Data SAP\04. OT y Av'

#Costos 
CeCo = r'BD-CeCo1004.xlsx' #Estatico
ClaseCostos = r'BD-ClaseCoste1004.xlsx' #Estatico
Budget = r'Presupuesto1004-2019Qlik.xlsx' #Estatico

filename3611 = r'3611.XLSX' #Dinamico
filename2993 = r'2993.XLSX' #Dinamico

#OT
filenameOt = r'IW39.xlsx'

In [341]:
#Charging DataSet
fullpath_CeCo = os.path.join(filepathCostos,CeCo)
fullpath_ClCo = os.path.join(filepathCostos,ClaseCostos)
fullpath_Budget = os.path.join(filepathCostos,Presupuesto)

fullpath_3611 = os.path.join(filepathCostos,filename3611)
fullpath_2993 = os.path.join(filepathCostos,filename2993)
fullpath_Ot = os.path.join(filepathOtAv,filenameOt)

In [342]:
fullpath_Budget

'\\\\u-srvnas\\Areas\\Mantenimiento\\00.1 Mantto Planeamiento\\0000. Gestión 2019\\1. REPORTES\\1. BASES DE DATOS DIARIOS\\Data SAP\\01. Costos\\Presupuesto1004-2019Qlik.xlsx'

In [351]:
dbceco = pd.read_excel(fullpath_CeCo, converters={'Centro de Costo':str})
dbclco = pd.read_excel(fullpath_ClCo, converters={'Clase de Coste':str})
dbbudget = pd.read_excel(fullpath_Budget, converters={'CeCo':str, 'Clase  de Coste':str})

In [207]:
db3611 = pd.read_excel(fullpath_3611, converters={'Orden partner':str,'Centro de coste':str,'Clase de coste':str})

In [208]:
db2993 = pd.read_excel(fullpath_2993, converters={'Orden':str,'Clase de coste':str})

In [285]:
dbot = pd.read_excel(fullpath_Ot, head = 0,converters={'Orden':str,'Aviso':str,'Centro coste':str})

In [354]:
dbbudget.dtypes


Escenario                      object
CeCo                           object
Descripción CeCo               object
Division                       object
PG.Proceso                     object
Año                             int64
Mes                            object
Grupo Naturaleza               object
Clase de Coste                  int64
Descripción Clase de Coste     object
Importe US$                   float64
dtype: object

In [324]:
#Reporte de Costos / 2993 <-> Clase de Coste
clco2993 = pd.merge(db2993[['Orden', 'Fe.contabilización','Valor/Moneda objeto', 'Clase de coste']], 
                    dbclco[['Clase de Coste', 'Grupo Naturaleza']], 
                    left_on='Clase de coste', right_on='Clase de Coste', 
                    how = 'left',
                    validate = 'm:1')
clco2993 = clco2993.drop(['Clase de coste','Clase de Coste'], axis = 1)

In [333]:
clco2993otceco.dtypes

Orden                          object
Fe.contabilización     datetime64[ns]
Valor/Moneda objeto           float64
Grupo Naturaleza               object
Clase de orden                 object
Centro de Costo                object
Área Responsable               object
Proceso                        object
dtype: object

In [326]:
#Reporte de Costos / 2993 <-> Orden de Mantenimiento
clco2993ot = pd.merge(clco2993, 
                    dbot[['Orden', 'Clase de orden','Centro coste']], 
                    on = 'Orden', 
                    how = 'left',
                    validate = 'm:1')

In [334]:
clco2993otceco = pd.merge(clco2993ot, 
                    dbceco[['Centro de Costo','Área Responsable','Proceso']],
                    left_on='Centro coste', right_on='Centro de Costo',
                    how = 'left',
                    validate = 'm:1')
clco2993otceco = clco2993otceco.drop(['Centro coste'], axis = 1)

In [247]:
RC01 = pd.merge(db2993[['Orden', 'Fe.contabilización', 'Valor/Moneda objeto', 'Clase de coste']],
                dbOt[['Orden', 'Clase de orden', 'Status sistema', 'Grupo planif.', 'Equipo']],
                on = ['Orden','Orden'],
                how = 'left')

In [308]:
clco2993otceco.dtypes

Orden                          object
Fe.contabilización     datetime64[ns]
Valor/Moneda objeto           float64
Grupo Naturaleza               object
Clase de orden                 object
Centro coste                   object
Centro de Costo                object
Área Responsable               object
Proceso                        object
dtype: object

In [296]:
dbceco.dtypes

Centro de Costo     object
CECO Descripción    object
Área Responsable    object
Proceso             object
Sub Proceso         object
Sub Grupo           object
Área (Gestor)       object
Zona                object
CECO Nombre         object
dtype: object

In [309]:
with pd.ExcelWriter('output.xlsx', mode='w') as writer:
    clco2993otceco.to_excel(writer, sheet_name='Sheet_name_1',index = False)

In [112]:
db2993.dtypes

División                                 float64
Centro                                   float64
Ejercicio                                float64
Período                                  float64
Fe.contabilización                datetime64[ns]
Orden                                     object
Denominación del objeto                   object
Valor/Moneda objeto                      float64
Moneda del objeto                         object
Material                                 float64
Texto breve de material                   object
Cantidad total reg.                      float64
Ud. cantidad contab.                      object
Documento compras                        float64
Texto de pedido                           object
Clase de coste                            object
Descrip.clases coste                      object
Proveedor                                float64
Nombre                                    object
Clase de documento                        object
Anulado             